In [15]:
from __future__ import print_function
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2
import random
import os
from model import Generator, Discriminator
import tensorflow_datasets as tfds

In [11]:
def load(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_png(image, channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [286, 286], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    return image


def load_image(image_file):
    image = load(image_file)

    return image


train_input_dataset_path = os.listdir('/home/naoki/Documents/GAN_practice/data/train/')
train_input_dataset_path = [os.path.join(os.path.abspath('/home/naoki/Documents/GAN_practice/data/train/'), p)
                            for p in train_input_dataset_path]

test_input_dataset_path = os.listdir('/home/naoki/Documents/GAN_practice/data/test/')
test_input_dataset_path = [os.path.join(os.path.abspath('/home/naoki/Documents/GAN_practice/data/test/'), p)
                           for p in test_input_dataset_path]

train_aug_dataset_path = os.listdir('/home/naoki/Documents/GAN_practice/data/aug/train')
train_aug_dataset_path = [os.path.join(os.path.abspath('/home/naoki/Documents/GAN_practice/data/aug/train'), p)
                          for p in train_aug_dataset_path]

test_aug_dataset_path = os.listdir('/home/naoki/Documents/GAN_practice/data/aug/test')
test_aug_dataset_path = [os.path.join(os.path.abspath('/home/naoki/Documents/GAN_practice/data/aug/test'), p)
                         for p in test_aug_dataset_path]

train_input_dataset = tf.data.Dataset.from_tensor_slices(train_input_dataset_path)
train_input_dataset = train_input_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

test_input_dataset = tf.data.Dataset.from_tensor_slices(test_input_dataset_path)
test_input_dataset = test_input_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_aug_dataset = tf.data.Dataset.from_tensor_slices(train_aug_dataset_path)
train_aug_dataset = train_aug_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

test_aug_dataset = tf.data.Dataset.from_tensor_slices(test_aug_dataset_path)
test_aug_dataset = test_aug_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_dataset = tf.data.Dataset.zip((train_input_dataset, train_aug_dataset))
test_dataset = tf.data.Dataset.zip((test_input_dataset, test_aug_dataset))

In [3]:
generator = Generator()
optimizer = tf.keras.optimizers.Adam()
checkpoint_path = "/home/naoki/Documents/GAN_practice/training_checkpoints/"
latest = tf.train.latest_checkpoint(checkpoint_path)
print(latest)
# latest = "/home/naoki/Documents/NEW-GAME-classification/tf/path/0.ckpt-1"

/home/naoki/Documents/GAN_practice/training_checkpoints/ckpt-6


In [4]:
ckpt = tf.train.Checkpoint(optimizer=optimizer, generator=generator)
status = ckpt.restore(latest).expect_partial()

In [30]:
test_img = test_dataset.take(1)
test = tfds.as_numpy(test_img)
org_img = np.array([])
for ts in test:
    org_img = ts[0]
    cv2.imwrite("org.png", org_img)

In [47]:
train_input, train_target = np.array([]), np.array([])
for tr_input, tr_target in train_dataset.batch(1).shuffle(1):
    train_input = np.array(tf.squeeze(tr_input))
    train_target = np.array(tf.squeeze(tr_target))

cv2.imwrite("train_input.png", train_input)
cv2.imwrite("train_target.png", train_target)

True

In [31]:
org_img.shape

(286, 286, 3)

In [32]:
after_img = generator(org_img, training=False)

ValueError: Input 0 of layer sequential is incompatible with the layer: expected ndim=4, found ndim=3. Full shape received: [286, 286, 3]

In [ ]:
def fit(train_ds, epochs, test_ds):
    for epoch in range(epochs):
        start = time.time()

        # for example_input, example_target in test_ds.take(1)

        for n, (input_image, target) in enumerate(train_ds.batch(1).shuffle(1)):
            print('.', end='')
            if (n+1) % 100 == 0:


EPOCHS = 1
fit(train_dataset, EPOCHS, test_dataset)